# Bag of Words Implementation For Enron Emails
We want to know the quality when using ONLY bag of words as features. Later in "Identifying Fraud from Enron Email" we will try combining this with our previous features (which unfortunately requires us to vectorize again but on full dataset).

In [3]:
%%time
### data_dict preparation. In this section we do all the adjustments we
### made to dataset in "Identifying Fraud from Enron Email" document.

### Warning! running this script will take a really long time, not to mention that you will need a copy of 
### Enron's email addresses from Udacity's Into to Machine Learning course.
### Download this file instead (400+Mb) https://www.dropbox.com/s/6o79pqvxrp0xylv/enron_emails_all_words.pkl?dl=0

# Change this to where you want all words pickle document to be kept.
all_words_path = 'D:\\Projects\\data_science\\nanodegree_data_analyst\\intro_to_ml\\enron_emails_all_words.pkl'

import sys
import cPickle
import matplotlib.pyplot as plt
import numpy as np
sys.path.append("../tools/")

from feature_format import featureFormat, targetFeatureSplit
from tester import test_classifier, dump_classifier_and_data

### Load the dictionary containing the dataset
data_dict = cPickle.load(open("final_project_dataset.pkl", "r") )

data_dict_updated = data_dict
data_dict_updated['BELFER ROBERT']['deferred_income'] = -102500
data_dict_updated['BELFER ROBERT']['deferral_payments'] = 'NaN'
data_dict_updated['BELFER ROBERT']['expenses'] = 3285
data_dict_updated['BELFER ROBERT']['director_fees'] = 102500
data_dict_updated['BELFER ROBERT']['total_payments'] = 3285
data_dict_updated['BELFER ROBERT']['exercised_stock_options'] = 'NaN'
data_dict_updated['BELFER ROBERT']['restricted_stock'] = 44093
data_dict_updated['BELFER ROBERT']['restricted_stock_deferred'] = -44093
data_dict_updated['BELFER ROBERT']['total_stock_value'] = 'NaN'

data_dict_updated['BHATNAGAR SANJAY']['other'] = 'NaN'
data_dict_updated['BHATNAGAR SANJAY']['expenses'] = 137864
data_dict_updated['BHATNAGAR SANJAY']['director_fees'] = 'NaN'
data_dict_updated['BHATNAGAR SANJAY']['total_payments'] = 137864
data_dict_updated['BHATNAGAR SANJAY']['exercised_stock_options'] = 15456290
data_dict_updated['BHATNAGAR SANJAY']['restricted_stock'] = 2604490
data_dict_updated['BHATNAGAR SANJAY']['restricted_stock_deferred'] = -2604490
data_dict_updated['BHATNAGAR SANJAY']['total_stock_value'] = 15456290

data_dict_updated_zero = data_dict_updated
for key, value in data_dict_updated_zero.items():
    for item_key, item_value in value.items():
        if value[item_key] == 'NaN':
            data_dict_updated_zero[key][item_key] = 0
            
data_dict_fe = data_dict_updated_zero

for name, item in data_dict_fe.items():
    data_dict_fe[name]['from_this_person_to_poi_ratio'] = 0
    data_dict_fe[name]['from_poi_to_this_person_ratio'] = 0
    if item['from_messages'] > 0:
        data_dict_fe[name]['from_this_person_to_poi_ratio'] = float(item['from_this_person_to_poi']) / float(item['from_messages'])
    if item['to_messages'] > 0:
        data_dict_fe[name]['from_poi_to_this_person_ratio'] = float(item['from_poi_to_this_person']) / float(item['to_messages'])

features_list_fe = [
    'poi',
    'bonus',
    'salary',
    'deferral_payments',
    'deferred_income',
    'director_fees',
    'exercised_stock_options',
    'expenses',
    'from_poi_to_this_person_ratio',
    'from_this_person_to_poi_ratio',
    'loan_advances',
    'long_term_incentive',
    'other',
    'restricted_stock',
    'restricted_stock_deferred',
    'salary',
    'shared_receipt_with_poi'
]

import os
import sys
sys.path.append("../tools/")
from parse_out_email_text import parseOutText

word_data = []
key_data = []
email_types = []
poi_data = []
# temp_counter = 0

if (os.path.isfile(all_words_path)):
    all_words = cPickle.load(open(all_words_path, "r"))
    word_data, key_data, email_types, poi_data = zip(*all_words)
else:
    dirpath = 'emails_by_address/'
    filenames = next(os.walk(dirpath))[2]
    for key, value in data_dict_fe.items():
        if value['email_address'] != 0:
            paths = [
                dirpath + 'from_' + value['email_address'] + '.txt',
                dirpath + 'to_' + value['email_address'] + '.txt',
            ]
            for idx, path in enumerate(paths):
                if os.path.isfile(path):
                    print "reading", path, '...'
                    email_list = open(path, 'r')
                    for email_path in email_list:
                        ### only look at first 100 emails when developing
                        ### once everything is working, remove this line to run over full dataset
                        # temp_counter += 1
                        # if temp_counter < 100:
                        email_path = '..' + email_path[19:-2]
                        if os.path.isfile(email_path):
                            email = open(email_path, 'r')
                            text = parseOutText(email)
                            word_data.append(text)
                            key_data.append(key)
                            poi_data.append(data_dict_fe[key]['poi'])
                            if idx == 0:
                                email_types.append('from')
                            else:
                                email_types.append('to')
    all_words = zip(word_data, key_data, email_types, poi_data)
    cPickle.dump(all_words, open(all_words_path, "w") )

Wall time: 20.9 s


In [4]:
all_words[10000]

(u'skeannsf dear all further to the prc committe meet pleas find attach the first draft of the peer group map by function it was decid that each member would review the propos to move from 4 peer group to 3 for mid year the attach was complet some week ago by the hr lead in your busi unit and is base on a mckinsey studi request by jeff skill and steve kean under your review of the attach inform pleas focus on the final two page that show the strawman on how the function fell out under the propos peer group structur i have copi in the hr lead for your busi area they have been instrument in complet the map and will meet with you to provid ani specif inform you requir concern placement with your busi area i would point out that there doe appear to be some inconsist in function placement within the peer group for some busi unit it will be our primari focus at the next meet to resolv these issu your assist have been contact regard the time and date of our next meet under separ cover for our

In [5]:
%%time
random = 13

from sklearn import cross_validation
words_train, words_test, labels_train, labels_test = cross_validation.train_test_split(word_data, poi_data, test_size=0.1, random_state=random)

from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.8, min_df=0.1,
                             max_features=100, stop_words='english')
import cProfile

# Use cProfile if you need detailed time analysis
# cProfile.run('features_train = vectorizer.fit_transform(words_train)')
# cProfile.run('features_test  = vectorizer.transform(words_test)')
features_train = vectorizer.fit_transform(words_train)
features_test  = vectorizer.transform(words_test)

Wall time: 1min 37s


In [4]:
print features_train.shape

(221303, 100)


In [5]:
features_test.shape

(24590, 100)

In [6]:
fnames = vectorizer.get_feature_names()
print fnames[1:50]

[u'2001', u'addit', u'ani', u'ask', u'attach', u'avail', u'busi', u'california', u'cc', u'chang', u'comment', u'committe', u'compani', u'contact', u'continu', u'current', u'date', u'david', u'day', u'develop', u'direct', u'discuss', u'email', u'energi', u'enron', u'execut', u'follow', u'forward', u'friday', u'gas', u'group', u'help', u'includ', u'inform', u'issu', u'jeff', u'john', u'just', u'know', u'let', u'like', u'list', u'look', u'make', u'manag', u'mark', u'market', u'meet', u'messag']


In [7]:
print len(fnames)

100


In [8]:
def top_5_word_scores(key):
    idx = key_data.index(key)
    # Not the most straightforward code but this is how to convert a row
    # of sparse matrix into a list
    row = features_train[idx].todense()[0].tolist()[0]

    word_scores = zip(range(0, features_train.shape[1]), row)
    sorted_scores = sorted(word_scores, key=lambda t: t[1] * -1)[:5]
    #  [(129753, 0.3526242375705134), (117135, 0.34794669254575034), (170214, 0.3069845443039424), (117128, 0.22619103462072823), (91625, 0.2245133019602352)]
    print("List of top 5 word scores of %s's first message:" % key_data[idx])
    for score in sorted_scores:
        print("%s - %0.2f" % (fnames[score[0]], score[1]))
        
top_5_word_scores('METTS MARK')
print ""
top_5_word_scores('LAY KENNETH L')

List of top 5 word scores of METTS MARK's first message:
comment - 0.40
respons - 0.39
review - 0.38
john - 0.37
includ - 0.34

List of top 5 word scores of LAY KENNETH L's first message:
work - 1.00
2000 - 0.00
2001 - 0.00
addit - 0.00
ani - 0.00


In [9]:
%%time
from sklearn import tree
clf = tree.DecisionTreeClassifier(random_state=random)

# cProfile.run('clf.fit(features_train, labels_train)')
clf.fit(features_train, labels_train)

Wall time: 1min 15s


And the quality metrics as follows:

In [10]:
%%time

PERF_FORMAT_STRING = "\
\tAccuracy: {:>0.{display_precision}f}\tPrecision: {:>0.{display_precision}f}\t\
Recall: {:>0.{display_precision}f}\tF1: {:>0.{display_precision}f}\tF2: {:>0.{display_precision}f}"


pred = clf.predict(features_test)
from sklearn.metrics import precision_recall_fscore_support

precision, recall, fscore, support = precision_recall_fscore_support(labels_test, pred, average='binary')
f2 = (1+2.0*2.0) * precision*recall/(4*precision + recall)

accuracy = clf.score(features_test, labels_test)
print clf
print PERF_FORMAT_STRING.format(accuracy, precision, recall, fscore, f2, display_precision = 5)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            random_state=13, splitter='best')
	Accuracy: 0.85665	Precision: 0.65509	Recall: 0.25822	F1: 0.37042	F2: 0.29382
Wall time: 55 ms
